# Export to ONNX

In [ ]:
!pip install -q --upgrade accelerate transformers ftfy natsort
!pip install -q git+https://github.com/huggingface/diffusers
!pip install optimum["onnxruntime"]
# !wget https://rhods-public.s3.amazonaws.com/src/xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl -O xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl
# !pip install xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl

In [ ]:
!wget -O convert_stable_diffusion_checkpoint_to_onnx.py https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_stable_diffusion_checkpoint_to_onnx.py

In [ ]:
import os

VERSION = "custom"
OUTPUT_DIR = os.path.join(os.getcwd(), f"{VERSION}/weights/redhat-dog")
ONNX_OUTPUT_DIR = os.path.join(os.getcwd(), f"{VERSION}/onnx-redhat-dog")

In [ ]:
from natsort import natsorted
from glob import glob
import os

WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] OUTPUT_DIR={OUTPUT_DIR}")
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")


In [ ]:
from optimum.onnxruntime import ORTStableDiffusionPipeline
import torch

# model_id = "runwayml/stable-diffusion-v1-5"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# pipe = ORTStableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to(device)
# g_cuda = torch.Generator(device=device)
# g_cuda.seed()

model_id = WEIGHTS_DIR
pipeline = ORTStableDiffusionPipeline.from_pretrained(model_id, export=True)
prompt = "photo of a rhteddy dog"
image = pipeline(prompt).images[0]
pipeline.save_pretrained(ONNX_OUTPUT_DIR)
image

### Save to S3

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    s3 = boto3.client('s3')
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)

upload_directory_to_s3(ONNX_OUTPUT_DIR, f"text-to-image/{VERSION}/onnx-redhat-dog")